**Статистическая языковая модель**

Евгений Борисов <esborisov@sevsu.ru>

подбираем наиболее вероятное продолжение цепочки слов

In [1]:
import gzip
import requests
from bs4 import BeautifulSoup

In [2]:
url='http://lib.ru/NEWPROZA/LOBAS/taxisty.txt'
text = BeautifulSoup(requests.get(url).text).get_text()
with gzip.open('taxisty.txt.gz','wt') as f: f.write(text)

# with gzip.open('taxisty.txt.gz','rt') as f: text = f.read()

text = text[1030:-7261].strip() # выкидываем заголовок и хвост страницы 
print(f'символов:{len(text)}\n---------------\n'%())
print(text[:343])

символов:637930
---------------

. 
     Вы прилетели в  Нью-Йорк и  остановились  в  одном  из отелей, глядящих
окнами на Центральный парк.  Наутро по  приезде вы вышли из  отеля, вдохнули
полной грудью очищенный зеленью парка воздух  и,  взглянув на часы, --  пора
было начинать хлопотливый день, --  направились к  первому из  таксомоторов,
выстроившихся вереницей у подъез


In [3]:
# url='http://az.lib.ru/d/dostoewskij_f_m/text_0080.shtml'
# text = BeautifulSoup(requests.get(url).text).get_text()
# with gzip.open('dostoewskij.txt.gz','wt') as f: f.write(text)

# # with gzip.open('dostoewskij.txt.gz','rt') as f: text = f.read()

# text = text[2876:-664184].strip() # выкидываем заголовок и хвост страницы 
# print(f'символов:{len(text)}\n---------------\n'%())
# print(text[:355])

In [4]:
from nltk import __version__ as nltk_version
print('nltk version:',nltk_version)

nltk version: 3.8.1


In [5]:
from tqdm.auto import tqdm
from random import sample

from nltk.tokenize import sent_tokenize as nltk_sentence_split
from nltk.tokenize import word_tokenize as nltk_tokenize_word

sentences = [ 
    nltk_tokenize_word(s,language='russian') # разбиваем предложения на слова
    for s in tqdm(nltk_sentence_split(text,language='russian')) # режем текст на отдельные предложения
]

print('предложений: %i\n'%(len(sentences)))
display( sample(sentences,1) )

  0%|          | 0/6652 [00:00<?, ?it/s]

предложений: 6652



[['Двигаясь',
  'на',
  'север',
  'по',
  'Парк-Авеню',
  ',',
  'по',
  'направлению',
  'к',
  'Центральному',
  'вокзалу',
  ',',
  'я',
  'включил',
  '``',
  'НЕ',
  'РАБОТАЮ',
  "''",
  '.']]

In [6]:
# sentences = sentences[:1024] # ограничиваем датасет для ускорения процеса 

---

In [7]:
from itertools import chain
from collections import Counter

text_tokens = list(chain(*sentences)) # собираем все токены (слова) из текста
vocab_size = len(set(text_tokens)) # размер словаря

text_tokens_freq = Counter( text_tokens ) # оценка частоты использования слов

print('всего слов тексте: %i'%(len(text_tokens)))
print('размер словаря: %i'%(vocab_size))
text_tokens_freq.most_common()[:10] # наиболее частые токены

всего слов тексте: 117099
размер словаря: 24073


[(',', 10727),
 ('--', 4221),
 ('.', 4000),
 ('и', 2453),
 ('в', 2409),
 ('не', 2077),
 ('я', 1513),
 ('!', 1470),
 ("''", 1412),
 ('...', 1392)]

----

In [8]:
# from nltk.util import bigrams
from nltk.util import ngrams as nltk_ngrams

# вынимаем все n-gram из текста
ngram_len = 2 # работаем с биграммами
text_ngrams = [ ngram for s in sentences for ngram in nltk_ngrams(s,ngram_len) ]
print('количество n-gram: %i'%(len(set(text_ngrams))))
sample(text_ngrams,5)

количество n-gram: 72689


[('Кеннеди', "''"),
 ('брежневских', 'законов'),
 ('напоминали', 'мне'),
 ('еще', 'один'),
 ('прежде', 'с')]

In [9]:
# cчитаем частоту n-gram
text_ngrams_freq = Counter(text_ngrams)
sample( list(text_ngrams_freq.items()), 7)

[(('сандвичи', ','), 1),
 ((',', 'событие'), 1),
 (('пример', 'лучше'), 1),
 (('же', 'мне'), 3),
 (('у', 'Тома'), 2),
 (('``', 'кладовую'), 1),
 (('вами', 'и'), 1)]

In [10]:
text_ngrams_freq.most_common()[:10] # наиболее частые n-ngram

[((',', 'что'), 807),
 ((',', 'и'), 631),
 ((',', 'а'), 466),
 ((',', '--'), 420),
 ((',', 'как'), 384),
 ((':', '--'), 366),
 ((',', 'но'), 319),
 ((',', 'я'), 297),
 (("''", ','), 294),
 ((',', 'не'), 247)]

----

Оценка вероятностей совместного использования слов со сглаживанием Лапласа


$$
P(w_n|w_{n-1}) = \frac{ C(w_{n-1} w_{n}) +1 }{ C(w_{n-1}) + V }
$$

In [11]:
text_ngrams_prob = { # оценка совместного использования токенов
    ngram : (text_ngrams_freq[ngram]+1) / ( text_tokens_freq[ ngram[0] ] + vocab_size )
    for ngram in text_ngrams_freq 
}

display( text_ngrams_prob )

{('Вы', 'прилетели'): 0.0001243574863206765,
 ('прилетели', 'в'): 0.00012461059190031152,
 ('в', 'Нью-Йорк'): 0.00033985348538630015,
 ('Нью-Йорк', 'и'): 0.00016596821708642794,
 ('и', 'остановились'): 7.539772298876574e-05,
 ('остановились', 'в'): 8.306337735692334e-05,
 ('в', 'одном'): 0.00030209198701004454,
 ('одном', 'из'): 0.00029062525948683884,
 ('из', 'отелей'): 0.00012212000325653342,
 ('отелей', ','): 0.000124595066035385,
 (',', 'глядящих'): 5.747126436781609e-05,
 ('глядящих', 'окнами'): 8.307717869901138e-05,
 ('окнами', 'на'): 8.306682726253271e-05,
 ('на', 'Центральный'): 0.00011788745677459919,
 ('Центральный', 'парк'): 0.0002491694352159468,
 ('парк', '.'): 0.000124595066035385,
 ('Наутро', 'по'): 8.30702774547267e-05,
 ('по', 'приезде'): 0.0002445785097016142,
 ('приезде', 'вы'): 8.306337735692334e-05,
 ('вы', 'вышли'): 8.242664029014178e-05,
 ('вышли', 'из'): 0.00016612675471384668,
 ('из', 'отеля'): 0.0004070666775217781,
 ('отеля', ','): 0.00033174372797014305,
 (

In [12]:
# оценка вероятности новых ngram по статистике text_ngrams_prob
def get_ngram_prob(ngram,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq):
    if ngram in text_ngrams_prob: return text_ngrams_prob[ngram]
    token_0_freq = text_tokens_freq[ngram[0]] if ngram[0] in text_tokens_freq else 0.
    return 1./(token_0_freq+len(text_tokens_freq))

выбираем дополнение предложения с наибольшей вероятностью цепочки слов

$$
P(w_1,\ldots, w_n) = \prod_{k=1}^{n} P(w_k|w_k-1)
$$

In [13]:
from operator import mul
from functools import reduce 

# оценка предложения
def get_sentence_prob(
        sentence,
        text_ngrams_prob=text_ngrams_prob,
        text_tokens_freq=text_tokens_freq,
        ngram_len=ngram_len
    ):
    return reduce( mul, [ 
        get_ngram_prob(ngram,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq)
        for ngram in nltk_ngrams(sentence,ngram_len) 
    ] )


In [14]:
# оценка всех возможных продолжений предложения
def get_next_token_prob(sentence,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq):
    
    sentence_prob = get_sentence_prob( # оценка предложения
            sentence,
            text_ngrams_prob=text_ngrams_prob,
            text_tokens_freq=text_tokens_freq
        )
    
    token_next = { # оценки всех возможных продолжений
        ngram[1] : 
            sentence_prob*
              get_ngram_prob(ngram,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq)
        for ngram in text_ngrams_prob if ngram[0]==sentence[-1] 
    }
    
    return token_next

In [15]:
from collections import OrderedDict

def get_top_prob_token(tokens_prob,n=3):  # n наиболее вероятных продолжений
    kf = lambda k: tokens_prob[k]
    return OrderedDict([
        (key,tokens_prob[key])
        for key in sorted(tokens_prob, key=kf, reverse=True)[:n]
    ])

----

In [16]:
# генерируем продолжения

for sentence in sample(sentences,30):
    if len(sentence)<10: continue
     # берём начало предложения
    sentence_ = sentence[:-(len(sentence)//4)]
    
    # генерируем возможные продолжения
    
    # считаем верояности продолжений
    next_token_prob = get_next_token_prob(sentence_) 
    
    # выбираем наиболее вероятные
    top_prob_token = get_top_prob_token(next_token_prob)
       
    print( 
        ' '.join(sentence_)
        +  ' ... ' 
        + '{ '
        + ' | '.join(top_prob_token.keys())
        + ' }'
        + '\n'
    )

Наш дом , `` рай бедняков '' покидала первая ... { . | женщина | очередь }

Он запустил руку глубоко в карман своих широченных , советского производства штанов , долго шарил там ( видимо , ... { что | и | а }

Смысл своей жизни он видел в этом предстоящем ... { открытии }

Мои же нынешние приятели , таксисты , не умели этого , как я не умел ... { командовать | ездить | приготовить }

Ты `` увидься '' со мной сейчас ! ... { .. | '' | `` }

Я имел полное право не впустить его и вообще вытурить всю гол-компанию из машины : в чекере не ... { было | мог | в }

Ибо какой еще выдающийся контрразведчик требовался для дознания по делу , основное обвинение по которому именно в том и заключалось , что преступник не желал скрывать свои преступные мысли : `` Ваша конституция ... { -- }

Ты мог получить `` Кеннеди '' , а поехал ... { в | ? | ... }

-- Ну , подумай сам : зачем я тебе ... { не | , | пригодятся }

-- Конечно ... -- Не верю ни одному ... { из | . | `` }

Белые водители , черные водители